In [3]:
import pandas as pd
import random
import json
from pandas.io.json import json_normalize #package for flattening json in pandas df

## Using Gutenberg Poetry dataset

In [ ]:
!head -n5 './data/gutenberg-poetry-v001.ndjson'

In [ ]:
p = 0.005  # 0.5% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(
    './data/gutenberg-poetry-v001.ndjson',
    names=['text'], 
    skiprows=lambda i: i>0 and random.random() > p,
    sep='\n',
#    columns=['text']
)

df = json_normalize(df['text'].apply(json.loads))
text_col = 's'
print(df.shape)

## Using Lyricsfreak.com dataset (from kaggle, under CreativeCommons)

In [4]:
!head -n10 ./data/songdata.csv

artist,song,link,text
ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face  
And it means something special to me  
Look at the way that she smiles when she sees me  
How lucky can one fellow be?  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?  


In [5]:
p = 0.05  # 5% of the lines
df = pd.read_csv(
    './data/songdata.csv',
    skiprows=lambda i: i>0 and random.random() > p,
)

df['text'] = df['text'].str.replace('\n', '')
text_col = 'text'
print(df.shape)

(2837, 4)


In [6]:
df.head()

,artist,song,link,text
0,ABBA,Fernando,/a/abba/fernando_20002896.html,Can you hear the drums Fernando? I remember l...
1,ABBA,Gimme Gimme Gimme,/a/abba/gimme+gimme+gimme_10001506.html,Half past twelve And I'm watching the late sh...
2,ABBA,Happy Hawaii,/a/abba/happy+hawaii_20002727.html,"Agnetha, Frida Early this morning I drove in t..."
3,ABBA,I Am Just A Girl,/a/abba/i+am+just+a+girl_20002700.html,"I am just a girl One among the others, nothin..."
4,ABBA,If It Wasn't For The Nights,/a/abba/if+it+wasnt+for+the+nights_20003038.html,"I got appointments, work I have to do Keepin'..."


In [11]:
from bert_serving.client import BertClient

bc = BertClient(ignore_all_checks=False)

KeyboardInterrupt: 

In [ ]:
%%time
encodings = bc.encode(df[text_col].values.tolist())

In [ ]:
encodings.shape

In [ ]:
#entry = 'The sky is blue and warm'
#entry = 'Nothing matters more !'
#entry = 'Shadow of my dream'
#entry = 'How are you doing ?'
#entry = 'Have a good day !'
#entry = 'Walk into the moon'
#entry = 'Come on stranger'
#entry = 'A world full of fantasy'
entry = 'Stars are falling'

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

enc_test = bc.encode([entry])

scores = cosine_similarity(enc_test, encodings)

top = 7
idxs = np.argsort(scores)[0][-top:]
idxs = np.flip(idxs)

In [ ]:
## Show similar lyrics/songs
print(scores[0][idxs])

df_tmp = pd.concat([df.iloc[idxs].reset_index(), pd.DataFrame(scores[0][idxs], columns=['score'])], axis=1)
df_tmp.head(20)